In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

In [4]:
metadata_df = pd.read_csv("data/network_metadata.tab", sep="\t")
metadata_df.loc[:, "IMDB_id"] = metadata_df["IMDB_id"].str[2:].astype('int')
metadata_df.head()

,GexfID,Title,IMDB_id,ReleaseDate,Slug,WeightedDegree,Modularity,PathLength,Diameter,ClusteringCoefficient,ConnectedComponents,Density,Edges,Characters
0,1,10 Things I Hate About You,147800,1999,10-Things-I-Hate-About-You,"13,040","0,194","1,779","3,000","0,567","3,000","0,233",225,25
1,2,12,488478,2007,12,"4,227","0,517","2,170","3,000","0,610","6,000","0,077",102,42
2,3,Twelve and Holding,417385,2005,Twelve-and-Holding,"7,333","0,473","2,449","4,000","0,593","2,000","0,140",139,26
3,5,127 Hours,1542344,2010,127-Hours,"3,833","0,160","1,722","2,000","0,339","4,000","0,152",33,11
4,6,1492: Conquest of Paradise,103594,1992,1492:-Conquest-of-Paradise,"7,941","0,350","2,043","4,000","0,705","1,000","0,153",164,34


In [5]:
imdb_data_df = pd.read_json("data/gexf_imdb_metadata.json", orient='index')
imdb_data_df

,movie_id,genres,runtime_min,box_office,imdb_rating,imdb_votes,directors,writers,character_actor_mapping
1,147800,"[Comedy, Drama, Romance]",97.0,"{'Budget': '$30,000,000 (estimated)', 'Opening...",7.3,279031.0,[Gil Junger],"[Karen McCullah, Kirsten Smith, William Shakes...",{'Bartender': {'BARTENDER': 'Carlos Lacamara'}...
2,488478,"[Crime, Drama, Thriller]",159.0,"{'Budget': '$4,000,000 (estimated)', 'Opening ...",7.7,13437.0,[Nikita Mikhalkov],"[Nikita Mikhalkov, Aleksandr Novototskiy-Vlaso...",{'Dochka svidetelya': {'DOC': 'Lyubov Rudneva'}}
3,417385,[Drama],94.0,"{'Budget': '$400,000 (estimated)', 'Opening We...",7.5,6550.0,[Michael Cuesta],[Anthony Cipriano],"{'Priest': {'PRIEST': 'Robertson Carricart'}, ..."
5,1542344,"[Biography, Drama]",94.0,"{'Budget': '$18,000,000 (estimated)', 'Opening...",7.6,333048.0,[Danny Boyle],"[Danny Boyle, Simon Beaufoy, Aron Ralston]","{'Aron's Mom': {'MOM': 'Kate Burton'}, 'Report..."
6,103594,"[Adventure, Biography, Drama, History]",154.0,"{'Budget': '$47,000,000 (estimated)', 'Cumulat...",6.5,26854.0,[Ridley Scott],[Rose Bosch],{}
...,...,...,...,...,...,...,...,...,...
910,1068680,"[Comedy, Romance]",104.0,"{'Budget': '$70,000,000 (estimated)', 'Opening...",6.8,319225.0,[Peyton Reed],"[Nicholas Stoller, Jarrad Paul, Andrew Mogel, ...",{'Father at Homeless Shelter': {'FATHER': 'Mik...
911,203230,[Drama],111.0,"{'Budget': '$1,200,000 (estimated)', 'Opening ...",7.5,26652.0,[Kenneth Lonergan],[Kenneth Lonergan],{}
912,128853,"[Comedy, Drama, Romance]",119.0,"{'Budget': '$65,000,000 (estimated)', 'Opening...",6.6,183169.0,[Nora Ephron],"[Miklós László, Nora Ephron, Delia Ephron]",{'T.V. Reporter': {'REPORTER': 'Nina Zoie Lam'}}
913,403702,"[Comedy, Drama, Romance]",90.0,"{'Budget': '$18,000,000 (estimated)', 'Opening...",6.4,71077.0,[Miguel Arteta],"[Gustin Nash, C.D. Payne]",{}


In [6]:
df = pd.merge(metadata_df, imdb_data_df, left_on="IMDB_id", right_on="movie_id")
df.loc[:,'directors-writers'] = df.loc[:,'directors'] + df.loc[:,'writers']
for i in range(len(df)):
    df.loc[:,'directors-writers'][i] = list(set(df.loc[:,'directors-writers'][i]))
df

C:\Users\Robin McCann\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,GexfID,Title,IMDB_id,ReleaseDate,Slug,WeightedDegree,Modularity,PathLength,Diameter,ClusteringCoefficient,ConnectedComponents,Density,Edges,Characters,movie_id,genres,runtime_min,box_office,imdb_rating,imdb_votes,directors,writers,character_actor_mapping,directors-writers
0,1,10 Things I Hate About You,147800,1999,10-Things-I-Hate-About-You,"13,040","0,194","1,779","3,000","0,567","3,000","0,233",225,25,147800,"[Comedy, Drama, Romance]",97.0,"{'Budget': '$30,000,000 (estimated)', 'Opening...",7.3,279031.0,[Gil Junger],"[Karen McCullah, Kirsten Smith, William Shakes...",{'Bartender': {'BARTENDER': 'Carlos Lacamara'}...,"[Gil Junger, Kirsten Smith, Karen McCullah, Wi..."
1,2,12,488478,2007,12,"4,227","0,517","2,170","3,000","0,610","6,000","0,077",102,42,488478,"[Crime, Drama, Thriller]",159.0,"{'Budget': '$4,000,000 (estimated)', 'Opening ...",7.7,13437.0,[Nikita Mikhalkov],"[Nikita Mikhalkov, Aleksandr Novototskiy-Vlaso...",{'Dochka svidetelya': {'DOC': 'Lyubov Rudneva'}},"[Vladimir Moiseenko, Aleksandr Novototskiy-Vla..."
2,3,Twelve and Holding,417385,2005,Twelve-and-Holding,"7,333","0,473","2,449","4,000","0,593","2,000","0,140",139,26,417385,[Drama],94.0,"{'Budget': '$400,000 (estimated)', 'Opening We...",7.5,6550.0,[Michael Cuesta],[Anthony Cipriano],"{'Priest': {'PRIEST': 'Robertson Carricart'}, ...","[Michael Cuesta, Anthony Cipriano]"
3,5,127 Hours,1542344,2010,127-Hours,"3,833","0,160","1,722","2,000","0,339","4,000","0,152",33,11,1542344,"[Biography, Drama]",94.0,"{'Budget': '$18,000,000 (estimated)', 'Opening...",7.6,333048.0,[Danny Boyle],"[Danny Boyle, Simon Beaufoy, Aron Ralston]","{'Aron's Mom': {'MOM': 'Kate Burton'}, 'Report...","[Aron Ralston, Simon Beaufoy, Danny Boyle]"
4,6,1492: Conquest of Paradise,103594,1992,1492:-Conquest-of-Paradise,"7,941","0,350","2,043","4,000","0,705","1,000","0,153",164,34,103594,"[Adventure, Biography, Drama, History]",154.0,"{'Budget': '$47,000,000 (estimated)', 'Cumulat...",6.5,26854.0,[Ridley Scott],[Rose Bosch],{},"[Rose Bosch, Ridley Scott]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,910,Yes Man,1068680,2008,Yes-Man,"9,143","0,351","1,899","2,000","0,729","2,000","0,097",308,51,1068680,"[Comedy, Romance]",104.0,"{'Budget': '$70,000,000 (estimated)', 'Opening...",6.8,319225.0,[Peyton Reed],"[Nicholas Stoller, Jarrad Paul, Andrew Mogel, ...",{'Father at Homeless Shelter': {'FATHER': 'Mik...,"[Nicholas Stoller, Andrew Mogel, Jarrad Paul, ..."
769,911,You Can Count on Me,203230,2000,You-Can-Count-on-Me,"11,400","0,275","1,752","3,000","0,583","4,000","0,211",153,18,203230,[Drama],111.0,"{'Budget': '$1,200,000 (estimated)', 'Opening ...",7.5,26652.0,[Kenneth Lonergan],[Kenneth Lonergan],{},[Kenneth Lonergan]
770,912,You've Got Mail,128853,1998,You've-Got-Mail,"8,769","0,374","2,131","4,000","0,727","2,000","0,130",218,36,128853,"[Comedy, Drama, Romance]",119.0,"{'Budget': '$65,000,000 (estimated)', 'Opening...",6.6,183169.0,[Nora Ephron],"[Miklós László, Nora Ephron, Delia Ephron]",{'T.V. Reporter': {'REPORTER': 'Nina Zoie Lam'}},"[Nora Ephron, Delia Ephron, Miklós László]"
771,913,Youth in Revolt,403702,2009,Youth-in-Revolt,"11,000","0,287","1,916","4,000","0,640","2,000","0,145",276,34,403702,"[Comedy, Drama, Romance]",90.0,"{'Budget': '$18,000,000 (estimated)', 'Opening...",6.4,71077.0,[Miguel Arteta],"[Gustin Nash, C.D. Payne]",{},"[Miguel Arteta, Gustin Nash, C.D. Payne]"


In [32]:
# A graph where nodes are movie titles and are conneted if they share a director or a writer.
Movies_Graph = nx.Graph()
titles = df.loc[:,'Title']
for title in titles:
    Movies_Graph.add_node(title)

    
# Add node if two movies share a director or a writer
# Edge weight = number of directors/writers in common 
for i in range(len(df)):
    for j in range(i+1, len(df)):
        title_i = titles[i]
        title_j = titles[j]
        dirwri_i = df.loc[i,'directors-writers']
        dirwri_j = df.loc[j,'directors-writers']
        intersection = set.intersection(set(dirwri_i), set(dirwri_j))
        if intersection:
            Movies_Graph.add_edge(title_i, title_j, weight=len(intersection))

In [34]:
# make subset of G that is ~50 nodes
Movies_Graph_subset = copy.deepcopy(Movies_Graph)
i=0
for node in Movies_Graph:
    if Movies_Graph.degree(node) < 12:
        Movies_Graph_subset.remove_node(node)

# print(len(Movies_Graph))
# print(len(Movies_Graph_subset))

773


In [14]:
## A graph where nodes are directors or writers and are connected if they have collaborated on a movie

# Get all unique directors and writers
writer_movies_map = {}
Writers_Graph = nx.Graph()
Directors_writers_unique = set()
for dw_list in df.loc[:,'directors-writers']:
    for dw in dw_list:
        Directors_writers_unique.add(dw)
        
# Add all writer nodes and make dictionary for every movie worked on by said writer
for dw in Directors_writers_unique:
    writer_movies_map[dw] = []
    Writers_Graph.add_node(dw)
    for i in range(len(df)):
        if dw in df.loc[i,'directors-writers']:
            writer_movies_map.get(dw).append(df.loc[i,'Title'])

In [21]:
writers = list(writer_movies_map.keys())

# Add edges between writers with weight equal to the number of movies they have collaborated on
for i in range(len(writers)):
    for j in range(i+1, len(writers)):
        writer_i = writers[i]
        writer_j = writers[j]
        movies_i = writer_movies_map[writer_i]
        movies_j = writer_movies_map[writer_j]
        intersection = set.intersection(set(movies_i), set(movies_j))
        if intersection:
            Writers_Graph.add_edge(writer_i, writer_j, weight=len(intersection))

In [33]:
# print(len(Writers_Graph))

1461


In [27]:
def draw_connection_with_edges(G, min_weight=None, max_weight=None, figsize=None, layout=None,
                               c=100, font_size=16, font_color='orange'):
    if figsize is not None:
        plt.figure(figsize=figsize)
        
    pos = layout or nx.random_layout(G)
    
    weights = {(u, v): d["weight"] for (u, v, d) in G.edges(data=True)}
    if min_weight is not None:
        weights = {k: v for k, v in weights.items() if v > min_weight}
    if max_weight is not None:
        weights = {k: v for k, v in weights.items() if v <= max_weight}
    filtered_edges = list(weights.keys())
    
    t = np.array(list(weights.values()))
    normalized_weights = c*t/t.sum()

    nx.draw(G, pos=pos, edgelist=filtered_edges, font_color=font_color, width=normalized_weights,
            with_labels=True, font_weight='bold', font_size=font_size)

    nx.draw_networkx_edge_labels(G,
                                 pos=pos,
                                 edge_labels=weights)
    plt.show()

In [28]:
# draw_connection_with_edges(Writers_Graph, min_weight=2, c=10, font_size=10)